![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_DistilBertForQuestionAnswering.ipynb)

## Import ONNX DistilBertForQuestionAnswering models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `DistilBertForQuestionAnswering` is only available since in `Spark NLP 5.1.3` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import DistilBERT models trained/fine-tuned for question answering via `DistilBertForQuestionAnswering` or `TFDistilBertForQuestionAnswering`. These models are usually under `Question Answering` category and have `distilbert` in their labels
- Reference: [TFDistilBertForQuestionAnswering](https://huggingface.co/transformers/model_doc/distilbert#transformers.TFDistilBertForQuestionAnswering)
- Some [example models](https://huggingface.co/models?filter=distilbert&pipeline_tag=question-answering)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- Albert uses SentencePiece, so we will have to install that as well

In [ ]:
!pip install -q --upgrade transformers[onnx]==4.48.3 optimum onnx

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [distilbert-base-cased-distilled-squad](https://huggingface.co/distilbert-base-cased-distilled-squad) model from HuggingFace as an example and load it as a `ORTModelForQuestionAnswering`, representing an ONNX model.
- In addition to the DistilBERT model, we also need to save the `DistilBertTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from transformers import DistilBertTokenizer
from optimum.onnxruntime import ORTModelForQuestionAnswering

MODEL_NAME = 'distilbert-base-cased-distilled-squad'
ONNX_MODEL = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForQuestionAnswering.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(ONNX_MODEL)

tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(ONNX_MODEL)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

('onnx_models/distilbert-base-cased-distilled-squad/tokenizer_config.json',
 'onnx_models/distilbert-base-cased-distilled-squad/special_tokens_map.json',
 'onnx_models/distilbert-base-cased-distilled-squad/vocab.txt',
 'onnx_models/distilbert-base-cased-distilled-squad/added_tokens.json')

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {ONNX_MODEL}

total 255024
-rw-r--r-- 1 root root       569 Jun 13 03:42 config.json
-rw-r--r-- 1 root root 260907945 Jun 13 03:42 model.onnx
-rw-r--r-- 1 root root       125 Jun 13 03:42 special_tokens_map.json
-rw-r--r-- 1 root root      1279 Jun 13 03:42 tokenizer_config.json
-rw-r--r-- 1 root root    213450 Jun 13 03:42 vocab.txt


- As you can see, we need to move `vocab.txt` from the tokenizer to `assets` folder which Spark NLP will look for

In [4]:
!mkdir {ONNX_MODEL}/assets && mv {ONNX_MODEL}/vocab.txt {ONNX_MODEL}/assets

In [5]:
!ls -lR {ONNX_MODEL}

onnx_models/distilbert-base-cased-distilled-squad:
total 254816
drwxr-xr-x 2 root root      4096 Jun 13 03:42 assets
-rw-r--r-- 1 root root       569 Jun 13 03:42 config.json
-rw-r--r-- 1 root root 260907945 Jun 13 03:42 model.onnx
-rw-r--r-- 1 root root       125 Jun 13 03:42 special_tokens_map.json
-rw-r--r-- 1 root root      1279 Jun 13 03:42 tokenizer_config.json

onnx_models/distilbert-base-cased-distilled-squad/assets:
total 212
-rw-r--r-- 1 root root 213450 Jun 13 03:42 vocab.txt


Voila! We have our `vocab.txt` inside assets directory

## Import and Save DistilBertForQuestionAnswering in Spark NLP


- **Install and set up Spark NLP in Google Colab**
  - This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

- **Optional: Use the latest versions**
  - If you prefer to use the latest versions instead, you can install them with:
    ```bash
    !wget http://setup.johnsnowlabs.com/colab.sh -O - | bash
    ```
  - Note: The latest versions may introduce breaking changes, so you might need to adjust the code accordingly.


In [6]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 28.4 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [7]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `DistilBertForQuestionAnswering` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `DistilBertForQuestionAnswering` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [8]:
from sparknlp.annotator import DistilBertForQuestionAnswering

spanClassifier = DistilBertForQuestionAnswering.loadSavedModel(
      ONNX_MODEL,
      spark
      )\
    .setInputCols(["document_question",'document_context'])\
    .setOutputCol("answer")\
    .setCaseSensitive(True)\
    .setMaxSentenceLength(512)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [9]:
spanClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(ONNX_MODEL))

Let's clean up stuff we don't need anymore

In [10]:
!rm -rf {ONNX_MODEL}

Awesome 😎  !

This is your DistlBertForQuestionAnswering model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [11]:
! ls -l {ONNX_MODEL}_spark_nlp_onnx

total 254840
-rw-r--r-- 1 root root 260947894 Jun 13 03:46 distilbert_classification_onnx
drwxr-xr-x 3 root root      4096 Jun 13 03:46 fields
drwxr-xr-x 2 root root      4096 Jun 13 03:46 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny AlbertForQuestionAnswering model in Spark NLP 🚀 pipeline!

In [ ]:
from sparknlp.base import MultiDocumentAssembler
from sparknlp.annotator import DistilBertForQuestionAnswering
from pyspark.ml import Pipeline

document_assembler = MultiDocumentAssembler() \
    .setInputCols(["question", "context"]) \
    .setOutputCols(["document_question", "document_context"])

spanClassifier_loaded = DistilBertForQuestionAnswering.load(
    f"./{ONNX_MODEL}_spark_nlp_onnx"
).setInputCols(["document_question", "document_context"]) \
 .setOutputCol("answer")

pipeline = Pipeline().setStages([
    document_assembler,
    spanClassifier_loaded
])

example = spark.createDataFrame([
    ["What's my name?", "My name is Clara and I live in Berkeley."]
]).toDF("question", "context")

result = pipeline.fit(example).transform(example)
result.select("answer.result").show(1, False)

+-------+
|result |
+-------+
|[Clara]|
+-------+



That's it! You can now go wild and use hundreds of `DistilBertForQuestionAnswering` models from HuggingFace 🤗 in Spark NLP 🚀
